In [47]:
# chatgpt를 이용하여서 iab category 와 연관된 타이틀 만들기 데이터 
import pandas as pd
from tqdm import tqdm

iab_data = pd.read_excel('3.0_iab_taxonomy.xlsx')

iab_data['final'] = iab_data['Tier 1'] + ' ' + iab_data['Tier 2'] + ' ' + iab_data['Tier 3']
iab_data['final'] = iab_data['final'].str.strip()
word_list = range(1, 10)

# prompts example
# with this {topic Attractions Casinos & Gambling} make title with {5} words
import os 
import openai
openai.api_key = ''

# 모델 - GPT 3.5 Turbo 선택
model = "gpt-3.5-turbo"
import sqlite3

# Connect to SQLite database
conn = sqlite3.connect('iab_data.db')
c = conn.cursor()

# Create a table if it doesn't exist
c.execute('''CREATE TABLE IF NOT EXISTS titles
             (id INTEGER PRIMARY KEY AUTOINCREMENT, topic TEXT, title TEXT)''')


for topic, word_num in zip(tqdm(iab_data['final']), word_list):
    try:
        query = f'with this {topic} make title with {word_num * 5} words'
        print(query)

        # 메시지 설정하기
        messages = [
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": query}
        ]

        # ChatGPT API 호출하기
        response = openai.ChatCompletion.create(
            model=model,
            messages=messages,
            temperature=0.9
        )
        answer = response['choices'][0]['message']['content']
        title = answer.strip('"')
        stopic = topic.strip()  
        
        # 디비 저장 
        c.execute("INSERT INTO titles (topic, title) VALUES (?, ?)", (topic, title))
        conn.commit()
    except:
        print('error occured')
conn.close()


  0%|          | 0/703 [00:00<?, ?it/s]

with this Attractions make title with 5 words


  0%|          | 1/703 [00:01<16:22,  1.40s/it]

with this Attractions Amusement and Theme Parks make title with 10 words


  0%|          | 2/703 [00:02<15:37,  1.34s/it]

...

with this Attractions Historic Site and Landmark Tours make title with 25 words


  1%|          | 5/703 [00:09<24:44,  2.13s/it]

with this Attractions Malls & Shopping Centers make title with 30 words


  1%|          | 6/703 [00:12<27:39,  2.38s/it]

with this Attractions Museums & Galleries make title with 35 words
